In [1]:
import csv
import struct as st
import numpy as np
import keyboard
import idx2numpy
from PIL import Image
from numpy import linalg as LA
from numpy import matlib
import sounddevice as sd
import pickle as pkl
import ipdb;
from scipy.io import loadmat;
from IPython.display import Audio
import matplotlib.pyplot as plt
import random
import sys


Defining the activation function and it's derivative (Sigmoid in this case)

In [2]:
alpha=1;
def actFcn(u):
    return (1/(1+np.exp(-alpha*u)))


def actFcnDer(u):
    return (alpha*np.exp(-alpha*u)/(1+np.exp(-alpha*u))**2)

Defining the Feedforward function assuming just one hidden layer

In [3]:
def feedForward(u1,W2,W3):
    act2 = actFcn(W2.dot(u1)).reshape(-1,1);
    # ipdb.set_trace();
    act2_b = np.vstack([np.ones((1,act2.shape[1])), act2])
    return(actFcn(W3.dot(act2_b)));

Reading training and test data

In [4]:
train_img = idx2numpy.convert_from_file('train-images-idx3-ubyte');
test_img = idx2numpy.convert_from_file('t10k-images-idx3-ubyte');
train_lab = idx2numpy.convert_from_file('train-labels-idx1-ubyte');
test_lab = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte');
L_train = train_img.shape[0];
L_test = test_img.shape[0];
train_img = train_img.reshape(L_train,28*28)/255; #Normalizing pixels from 255 -> 1
test_img = test_img.reshape(L_test,28*28)/255; #Normalizing pixels from 255 -> 1
train_img_input = np.vstack([np.ones((1,L_train)), train_img.T]);
test_img_input = np.vstack([np.ones((1,L_test)), test_img.T]);
train_lab = train_lab.reshape(-1,1)
train_lab_vec = np.zeros(L_train*n3);
cols = np.arange(0,L_train,1);
cols = cols.reshape(-1,1);
train_lab_vec[10*cols + train_lab]=1;
train_lab_vec = (train_lab_vec.reshape(L_train,n3)).transpose();

Initializing Weigths and number of neurons in diff. layers of network

In [9]:
n1 = train_img.shape[1];
n3=10;
eta=5e-2;
tol = 6e-3;
err_cum_arr = 0;
maxIter=10
for n2 in np.arange(3,19,4):
    W2_b = np.random.rand(n2, n1+1)*1e-2;
    W2 = W2_b[:,1:];
    W3_b = np.random.rand(n3, n2+1)*1e-2;
    W3 = W3_b[:,1:];
    err_cum = tol*1e3;
    pred_acc_arr = 0;
    err_cum_prev=err_cum*1e3;
    train_samples = 60000;
    test_samples = 10000;
    iter=0;
    while err_cum>tol and iter<iterMax:# and err_cum_prev>err_cum:
        iter = iter+1;
        # err_cum_prev=err_cum;
        err_cum = 0.0;
        for i in range(0,train_samples):
            Delta2 = W2_b*0.0;
            Delta3 = W3_b*0.0;
            z2 = W2_b.dot(train_img_input[:,i].reshape(-1,1));
            z2_b = np.vstack([np.ones((1,z2.shape[1])), z2]);
            a2 = actFcn(z2);
            a2_b = np.vstack([np.ones((1,a2.shape[1])), a2]);

            z3 = W3_b.dot(a2_b);
            a3 = actFcn(z3);

            delta3 = np.multiply(a3 - train_lab_vec[:,i].reshape(-1,1),actFcnDer(z3))#if using MSE cost function
            # delta3 = -(train_lab_vec[:,i].reshape(-1,1) - a3);#if using log cost function from Andrew Ng

            delta2 = np.multiply(W3_b.T.dot(delta3), actFcnDer(z2_b));
            delta2 = np.delete(delta2,0,0);

            d3 = delta3.reshape(-1,1);
            # ipdb.set_trace()
            a2_b_vec = a2_b.reshape(-1,1);
            Delta3 = Delta3 + d3.dot(a2_b_vec.T);
            # ipdb.set_trace()

            d2 = delta2.reshape(-1,1);
            a1_b_vec = train_img_input[:,i].reshape(-1,1);
            # Delta2 = Delta2 + d2.dot(a1_b_vec.T)/train_samples;#if using cumulative grad descent
            Delta2 = Delta2 + d2.dot(a1_b_vec.T);#if using stochastic grad descent
            # err_cum = err_cum + np.mean(np.multiply(delta3,delta3));#if using log cost function from Andrew Ng
            W3_b = W3_b - eta*Delta3;
            W2_b = W2_b - eta*Delta2;
            err_cum = err_cum + np.mean(np.multiply((train_lab_vec[:,i].reshape(-1,1) - a3),(train_lab_vec[:,i].reshape(-1,1) - a3)));#if using MSE cost function
        err_cum = err_cum/train_samples;
        err_cum_arr = np.vstack([err_cum_arr,err_cum]);
        print("MSE on training data = ",err_cum)
        correct_pred_test=0;
        for i in range(0,test_samples):
            predicted_label = np.argmax(feedForward(test_img_input[:,i],W2_b,W3_b));
            actual_label = test_lab[i];
            if actual_label==predicted_label:
                correct_pred_test = correct_pred_test + 1;
        pred_acc_arr = np.vstack([pred_acc_arr, correct_pred_test]);
        print("Prediction acc. on test data",correct_pred_test)
    

MSE on training data =  0.04328578540299913
Prediction acc. on test data 9002
MSE on training data =  0.014186407232872187
Prediction acc. on test data 9218
MSE on training data =  0.012067437090434487
Prediction acc. on test data 9276
MSE on training data =  0.011108927886002522
Prediction acc. on test data 9310
MSE on training data =  0.010502445584247773
Prediction acc. on test data 9334
MSE on training data =  0.010070899882017712
Prediction acc. on test data 9347
MSE on training data =  0.009739125740381736
Prediction acc. on test data 9359
MSE on training data =  0.009466134452235812
Prediction acc. on test data 9368
MSE on training data =  0.009233508285191067
Prediction acc. on test data 9381
MSE on training data =  0.009033884689902884
Prediction acc. on test data 9390
MSE on training data =  0.008858078191169926
Prediction acc. on test data 9391
MSE on training data =  0.008700535663084484
Prediction acc. on test data 9392
MSE on training data =  0.00855953396359906
Predictio

Calculating prediction accuracy

In [1]:
err_cum_arr = np.delete(err_cum_arr,0,0)
pred_acc_arr = np.delete(pred_acc_arr,0,0)
plt.plot(err_cum_arr,'b')
plt.xlabel('No. of Iterations/passes')
plt.ylabel('MSE')
plt.title('Backpropagation on training data')
plt.rcParams.update({'font.size':25})
plt.tight_layout()
plt.savefig('./train_err_sigmoid.png')
plt.show()
#plt.close()

plt.plot(pred_acc_arr/test_samples*100,'g')
plt.xlabel('No. of passes on training data')
plt.ylabel('Test prediction accuracy')
plt.title('Test accuracy')
plt.rcParams.update({'font.size':12})
plt.tight_layout()
plt.savefig('./test_err_sigmoid.png')
plt.show()
#plt.close()



NameError: name 'np' is not defined